In [1]:


from rcm_library import *

from __future__ import division
import itertools
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import numpy as np
import pandas as pd
import math
from sklearn import metrics
from random import randint
from matplotlib import style
import seaborn as sns

from sklearn.cluster import DBSCAN
from sklearn.preprocessing import MinMaxScaler

from IPython.display import clear_output

from env import host, username, password


from pandas import value_counts

def get_db_url(database, host=host, user=username, password=password):
    return f'mysql+pymysql://{user}:{password}@{host}/{database}'




In [2]:
dfa = pd.read_table("anonymized-curriculum-access.txt", sep = '\s', header = None, 
                   names = ['date', 'time', 'page', 'id', 'cohort', 'ip'])




In [3]:
df=dfa# save point

In [4]:
df['datetime']=df.date +" "+ df.time

df['datetime']=pd.to_datetime(df['datetime'])


In [5]:



df.set_index('datetime',inplace=True)
df.drop(columns=['date','time'],inplace=True)
df.head(3)




,page,id,cohort,ip
datetime,,,,
2018-01-26 09:55:03,/,1,8.0,97.105.19.61
2018-01-26 09:56:02,java-ii,1,8.0,97.105.19.61
2018-01-26 09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61


In [6]:
schema='curriculum_logs'
url = f'mysql+pymysql://{env.username}:{env.password}@{env.host}/{schema}'



query_2 = f'''
SELECT table_name AS "Tables",
ROUND(((data_length + index_length) / 1024 / 1024), 4) AS "Size (MB)"
FROM information_schema.TABLES
WHERE table_schema = "{schema}"
ORDER BY (data_length + index_length) DESC;
'''

info2 = pd.read_sql(query_2, get_db_url(schema))

tablenames = [x[0] for x in [list(i) for i in info2.values]]

x=[(pd.read_sql(f'describe {x}', get_db_url(schema)))for x in tablenames]






display(
    f'In {schema} you have the following table names and their sizes:', info2)
[display(i) for i in x]





'In curriculum_logs you have the following table names and their sizes:'

,Tables,Size (MB)
0,logs,90.6250
1,cohorts,0.0313


,Field,Type,Null,Key,Default,Extra
0,date,text,YES,,None,
1,time,text,YES,,None,
2,path,text,YES,,None,
3,user_id,bigint,YES,,None,
4,cohort_id,double,YES,,None,
5,ip,text,YES,,None,


,Field,Type,Null,Key,Default,Extra
0,id,bigint,YES,MUL,None,
1,name,text,YES,,None,
2,slack,text,YES,,None,
3,start_date,text,YES,,None,
4,end_date,text,YES,,None,
5,created_at,text,YES,,None,
6,updated_at,text,YES,,None,
7,deleted_at,double,YES,,None,
8,program_id,bigint,YES,,None,


[None, None]

In [7]:


# query_logs = '''
# SELECT *
# FROM logs;
# '''
# logs = pd.read_sql(query_logs, url)




# query_cohorts = '''
# SELECT *
# FROM cohorts;
# '''
# cohorts = pd.read_sql(query_cohorts, url)


In [8]:

# sql_query = """
#                 select * from customers 
#                 join contract_types using (contract_type_id)
#                 join internet_service_types using (internet_service_type_id)
#                 join payment_types using (payment_type_id)
#                 """




def curriculum_logs_new():
    '''
    This function reads the curriculum data from the Codeup db into a df.
    '''
    schema='curriculum_logs'
    sql_query=f''' 

    select * from logs as lg
    left join cohorts as cht
    on
    lg.cohort_id = cht.id


    
    
    
    
    
    
    
    '''
    
    # Read in DataFrame from Codeup db.
    df = pd.read_sql(sql_query, get_db_url(schema))
    
    return df




def get_curriculum_logs():
    '''
    This function reads in telco data from Codeup database, writes data to
    a csv file if a local file does not exist, and returns a df.
    '''
    csvname='curriculum_logs.csv'
    if os.path.isfile(csvname):
        
        # If csv file exists read in data from csv file.
        df = pd.read_csv(csvname, index_col=0)
        
    else:
        
        # Read fresh data from db into a DataFrame
        df = curriculum_logs_new()
        
        # Cache data
        df.to_csv(csvname)
    
    return df















In [9]:
cur_logs=get_curriculum_logs()

In [10]:
cur_logs.memory_usage().sum() / (1024**2) 

In [11]:
cur_logs.head(5)

cur_logs[(cur_logs.id==cur_logs.cohort_id)]
cur_logs=cur_logs[(cur_logs.id==cur_logs.cohort_id)]




In [12]:
cur_logs

,date,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,8.0,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,NaN,1.0
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,22.0,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,NaN,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
900218,2021-04-21,16:41:51,jquery/personal-site,64,28.0,71.150.217.33,28.0,Staff,#,2014-02-04,2014-02-04,2018-12-06 17:04:19,2018-12-06 17:04:19,NaN,2.0
900219,2021-04-21,16:42:02,jquery/mapbox-api,64,28.0,71.150.217.33,28.0,Staff,#,2014-02-04,2014-02-04,2018-12-06 17:04:19,2018-12-06 17:04:19,NaN,2.0
900220,2021-04-21,16:42:09,jquery/ajax/weather-map,64,28.0,71.150.217.33,28.0,Staff,#,2014-02-04,2014-02-04,2018-12-06 17:04:19,2018-12-06 17:04:19,NaN,2.0
900221,2021-04-21,16:44:37,anomaly-detection/discrete-probabilistic-methods,744,28.0,24.160.137.86,28.0,Staff,#,2014-02-04,2014-02-04,2018-12-06 17:04:19,2018-12-06 17:04:19,NaN,2.0


In [13]:

programs=cur_logs[['program_id','cohort_id','start_date',"end_date",'name']]

grp1=programs.groupby(['name','cohort_id','start_date','end_date','program_id'])



In [14]:

metadata=pd.DataFrame(list(grp1.groups.keys()),columns=['name','cohort','start_date','end_date','program_id'])
metadata


,name,cohort,start_date,end_date,program_id
0,Andromeda,31.0,2019-03-18,2019-07-30,2.0
1,Apex,56.0,2020-02-24,2020-07-29,2.0
2,Apollo,9.0,2015-03-30,2015-07-29,4.0
3,Arches,1.0,2014-02-04,2014-04-22,1.0
4,Badlands,2.0,2014-06-04,2014-08-22,1.0
5,Bash,61.0,2020-07-20,2021-01-21,2.0
6,Bayes,34.0,2019-08-19,2020-01-30,3.0
7,Betelgeuse,32.0,2019-05-28,2019-10-08,2.0
8,Ceres,33.0,2019-07-15,2019-12-11,2.0
9,Curie,55.0,2020-02-03,2020-07-07,3.0


1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?
2. Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?
3. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?
4. Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP addresses?
5. At some point in 2019, the ability for students and alumni to access both curriculums (web dev to ds, ds to web dev) should have been shut off. Do you see any evidence of that happening? Did it happen before?
6. What topics are grads continuing to reference after graduation and into their jobs (for each program)?
7. Which lessons are least accessed? 


In [15]:

# grp1keys=tuple(grp1.groups.keys())
# for i in range(len(grp1keys)):
#     # print(grp1.get_group(grp1keys[i]).value_counts(normalize=False).nlargest(1).to_dict())
#     # display(pd.DataFrame(grp1.get_group(grp1keys[i])))
#     display(grp1.get_group(grp1keys[i]).to_dict())





In [16]:

cols=df.columns.to_list();cols



# df.page.


['page', 'id', 'cohort', 'ip']

In [192]:
## the / in page seems to be an error so I am droping those rows from the df for now.

df=df[df.page!='/']


grp1=programs.groupby(['name','cohort_id','start_date','end_date','program_id'])
metadata=pd.DataFrame(list(grp1.groups.keys()),columns=['name','cohort_id','start_date','end_date','program_id'])
typenamecohort=pd.DataFrame(list(metadata.groupby(['cohort_id','name','program_id']).groups.keys()),columns=['cohort_id','name','program_id'])
typenamecohort
df



,page,id,cohort,ip
datetime,,,,
2018-01-26 09:56:02,java-ii,1,8.0,97.105.19.61
2018-01-26 09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61
2018-01-26 09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61
2018-01-26 09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61
2018-01-26 09:56:41,javascript-i/loops,2,22.0,97.105.19.61
...,...,...,...,...
2021-04-21 16:41:51,jquery/personal-site,64,28.0,71.150.217.33
2021-04-21 16:42:02,jquery/mapbox-api,64,28.0,71.150.217.33
2021-04-21 16:42:09,jquery/ajax/weather-map,64,28.0,71.150.217.33


In [ ]:

set(df.cohort)
set(typenamecohort.cohort_id)

In [188]:



mergedlostdates=df.join(typenamecohort,how='left',on='cohort')
cohortnames=set(mergedlostdates.name.unique())
cohortnames=list(cohortnames-{nan})
cohortnames.pop(0) 
mergedlostdates=mergedlostdates.fillna(0) 
numcols=mergedlostdates.select_dtypes('number').columns.to_list()
mergedlostdates[numcols]=mergedlostdates[numcols].astype(int)


nomenclaturemap={3:'DS',2:'Full_Stack_PHP',1:'Full_Stack_PHP',4:'Front_End'}




mergedlostdates['program_name']=mergedlostdates['program_id'].map(nomenclaturemap)
a=list(set(mergedlostdates.cohort_id.to_list()))
b=list(set(mergedlostdates.cohort.to_list()))
a
b



In [151]:
cohortnames.sort()
cohortnames


['Andromeda',
 'Apex',
 'Apollo',
 'Badlands',
 'Bayes',
 'Betelgeuse',
 'Ceres',
 'Curie',
 'Denali',
 'Europa',
 'Fortuna',
 'Franklin',
 'Hampton',
 'Ike',
 'Joshua',
 'Lassen',
 'Mammoth',
 'Niagara',
 'Olympic',
 'Pinnacles',
 'Quincy',
 'Sequoia',
 'Staff',
 'Teddy',
 'Ulysses',
 'Wrangell',
 'Xanadu',
 'Yosemite',
 'Zion']

,name,program_id,map
datetime,,,
2018-01-26 09:56:02,Ike,1,WebDev
2018-01-26 09:56:05,Ike,1,WebDev
2018-01-26 09:56:06,Ike,1,WebDev
2018-01-26 09:56:24,Xanadu,2,WebDev
2018-01-26 09:56:41,Xanadu,2,WebDev
...,...,...,...
2021-04-21 16:41:51,Ceres,2,WebDev
2021-04-21 16:42:02,Ceres,2,WebDev
2021-04-21 16:42:09,Ceres,2,WebDev


In [180]:

a=nomenclature['map'].values.tolist()
b=nomenclature['program_id'].values.tolist()
mapper=dict(zip(b,a))
mapper

{1: 'WebDev', 2: 'WebDev', 0: nan, 4: 'WebDev', 3: 'DS'}

'Andromeda',
 'Apex',
 'Apollo',

 'Bayes',
 'Betelgeuse',
 'Ceres',
 
 'Denali',
 'Europa',
 'Fortuna',
 'Franklin',
 'Hampton',
 'Ike',
 'Joshua',
 'Lassen',
 'Mammoth',
 'Niagara',
 'Olympic',
 'Pinnacles',
 'Quincy',
 'Sequoia',
 'Staff',
 'Teddy',
 'Ulysses',
 'Wrangell',
 'Xanadu',
 'Yosemite',


In [153]:
#Question 1
# 1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?



program_traffic=mergedlostdates.groupby(['program_id']).page


question1=pd.DataFrame(program_traffic.agg([pd.Series.mode,pd.Series.nunique]));question1


,mode,nunique
program_id,,
0,search/search_index.json,1610
1,index.html,602
2,javascript-i,1895
3,toc,389
4,content/html-css,142


In [154]:
#Question 2
# 2. Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?




cohort_traffic=mergedlostdates.groupby(['program_id','cohort','name']).page
kwargs={'ascending':False}
question2=pd.DataFrame(cohort_traffic.agg([pd.Series.mode,pd.Series.nunique,value_counts],**kwargs));question2


mode  \
program_id cohort name                                                                                   
0          0      0                                                           search/search_index.json   
           51     0                                                                       javascript-i   
           52     0                                                                                toc   
           53     0                                                                                toc   
           55     0                                                            6-regression/1-overview   
           56     0                                                           search/search_index.json   
           57     0                                                           search/search_index.json   
           58     0                                                                                toc   
           59     0                                                            classification/overview   
           61     0                                                           search/search_index.json   
           62     0                                                                                toc   
           132    0                                                                       javascript-i   
           133    0                                           classification/scale_features_or_not.svg   
           134    0                                                                       javascript-i   
           135    0                                                                                toc   
           137    0                                             fundamentals/modern-data-scientist.jpg   
           138    0                                                                                toc   
           139    0                                                javascript-i/introduction/operators   
1          1      Badlands                                                                javascript-i   
           2      Denali                                                   content/php_ii/command-line   
           4      Franklin    [mkdocs/search_index.json, prework/databases, prework/versioning/github]   
           6      Hampton                                                            javascript-ii/es6   
           8      Ike                                                                         java-iii   
           9      Joshua                                                              content/html-css   
           11     Lassen                                                                      html-css   
           14     Olympic                                                                   index.html   
           16     Quincy                                                                        spring   
2          12     Mammoth                                                             content/html-css   
           13     Niagara                                                                   index.html   
           15     Pinnacles                                                                     java-i   
           17     Sequoia                                                                 javascript-i   
           18     Teddy                                                                   javascript-i   
           19     Ulysses                                                        content/laravel/intro   
           21     Wrangell                                                                      spring   
           22     Xanadu                                                                        spring   
           23     Yosemite                                                    mkdocs/search_index.json   
           24     Staff                                               

In [155]:
#Question 3


In [156]:
#Question 4
suspicious_ip=mergedlostdates.groupby(['id']).ip

question4=pd.DataFrame(suspicious_ip.agg([pd.Series.mode,pd.Series.nunique]))
question4=question4.sort_values(by='nunique',ascending=False)
question4=question4[question4['nunique']>=question4['nunique'].quantile(.99)];question4

,mode,nunique
id,,
228,97.105.19.58,148
843,172.58.99.111,122
533,205.251.148.178,91
690,67.11.40.170,86
530,97.105.19.58,86
226,97.105.19.58,83
460,89.187.164.151,81
590,130.45.47.56,74
127,97.105.19.61,61


In [157]:
# 5. At some point in 2019, the ability for students and alumni to access both curriculums (web dev to ds, ds to web dev) should have been shut off. Do you see any evidence of that happening? Did it happen before?

observed=pd.crosstab(mergedlostdates.program_id, mergedlostdates.page)
observed
# mergedlostdates[['name','page']]

# chi2, p, degf, expected = stats.chi2_contingency(observed)

# print('Observed\n')
# print(observed.values)
# print('---\nExpected\n')
# display(pd.DataFrame(expected))
# print('---\n')
# print(f'chi^2 = {chi2:.4f}')
# print(f'p     = {p:.4f}')

page,0,%20https://github.com/RaulCPena,',",%20https://github.com/RaulCPena",.git,.gitignore,.well-known/assetlinks.json,00_,00_index,01_intro,...,web-design/ui/visuals,web-design/ux,web-design/ux/layout,web-design/ux/layout/.json,web-design/ux/purpose,web-dev-day-two,where,working-with-time-series-data,wp-admin,wp-login
program_id,,,,,,,,,,,,,,,,,,,,,
0,1,1,0,1,0,1,2,8,20,44,...,130,1,153,0,140,2,1,14,0,0
1,0,0,0,0,0,0,0,0,0,0,...,6,0,7,0,8,0,0,0,0,0
2,0,0,1,0,1,1,0,1,9,9,...,84,0,91,0,89,0,0,1,1,1
3,0,0,0,0,0,0,0,0,0,0,...,35,0,35,1,29,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [158]:
#Question 7

leastaccesed=mergedlostdates.page.value_counts()
question7=pd.DataFrame(leastaccesed)

question7


,page
search/search_index.json,19519
javascript-i,18983
toc,18297
java-iii,13733
html-css,13635
...,...
12-distributed-ml/6.1-prepare-part-1,1
12-distributed-ml/6.4-prepare-part-4,1
4-python,1
4-python/overview,1


In [159]:



y=['.com','.py','.md','.json','.tex','.com','.jpeg','.jpg','.html','.ico','.pn','.png','.svg','.csv']
y=[i.lower().strip() for i in y]


a=list(question7.index)
a=[i.lower() for i in a]

AttributeError: 'int' object has no attribute 'lower'

In [ ]:

finalist=[]
for x in a:
    inotx=True
    for i in y:
             
        if i not in x:
            continue
        else:
            inotx=False
    if  inotx==True:
        finalist.append(x)

a=finalist


wordstogroupon=[\
'sql','cli',\
'appendix','javascript','java','jquery','stats','spring',\
'timeseries','python','classification','regression','nlp','clustering','sklearn','group','tidy','tree','storytelling',\
'relationships','capstone','storytelling','regex','anom','kmean','advanced','css','html','explore','prep',\
'ajax','git','matplot','prework','quiz','dbscan','php','set','model','distributed','bokeh','exploration',\
'tableau','seaborn','where','acquisition','spark','joins','distributions','knn','fundamentals','acquire','analysis','bio','merge','ml','effects',\
'wrangle','prophet','forest','subqueries','machinelearning','interview','polymorphism','loops','jupiter','pipeline','types','object',\
'method','instructors','job','grades','json','student','restful','index','bayes','npm','register','mvc','extra','documentation','elements','exceptions','scale',\
'mapbox','cli','eval','correlation','bom','web','team','functions','conditionals','data','passwords','professional','stat','glossary','tools','api','expressions','introduction']
wordstogroupon=list(set(wordstogroupon))

wordslists=[list() for i in wordstogroupon]


wordslist=dict(zip(wordstogroupon,wordslists))
popped=[]
for i in wordslist.keys():
    for x1,x in enumerate(a):
        if i in x:
            u=a.pop(x1)
            popped.append(u)
            wordslist.get(i).append(u)


        
        else:
            continue
a=list(set(a)-set(popped))
wordslist.update({'remainder':a})



wordslist.keys()
tosort=wordslist.get("remainder")



In [ ]:

nonalpha=str('1234567890~!%^&*/_-.:;  ')
nonalpha=[(f'{i}') for i in nonalpha]


bsmart=[]
for i in tosort:
    for x in nonalpha:
        i=i.replace(f'{x}','_')
    i=i.replace('_','')
    bsmart.append(i)
bsmart=list(set(bsmart))


smarterlist=[]

for i in bsmart:
    if len(i) <7:
           
        if len(i)==6:
             i=i[:5]

        elif len(i)>5:
            i=i[:4]
        elif len(i)>4:
            i=i[:3]
        elif len(i)>3:
            i=i[:3]
        else:
            continue
        smarterlist.append(i)


smarterlist=list(set(smarterlist))
smarterlist.sort()


In [ ]:

popped=[]
for i in wordslist.keys():
    for x1,x in enumerate(a):
        if i in x:
            u=a.pop(x1)
            popped.append(u)
            wordslist.get(i).append(u)


        
        else:
            continue
a=list(set(a)-set(popped))
wordslist.update({'remainder':a})



keyslist=list(wordslist.keys())

for i in keyslist:
    display(pd.DataFrame(wordslist.get(i),columns=[i]))

,group
0,mysql/group-by
1,sql/group-by
2,stats/compare-group-membership
3,3-sql/8-group-by
4,5-stats/4.4-compare-group-membership
5,3.7-group-by
6,group-by


,json
0,spring/extra-features/json-response
1,13-advanced-topics/3.8-json-responses
2,advanced-topics/json-responses
3,12.3.7_json_responses
4,json-responses
5,extra-features/json-response
6,12-advanced-topics/3.8-json-responses


,npm
0,javascript-ii/npm
1,npm


,documentation
0,appendix/documentation
1,content/appendix/documentation
2,documentation


,prep
0,classification/prep
1,7-classification/3-prep
2,regression/acquire-and-prep
3,6-regression/2.0-acquire-and-prep
4,9-timeseries/3-prep
5,11-nlp/4-prepare
6,timeseries/prep
7,6.4_prep
8,8.4_prep
9,nlp/prepare


...

,sklearn


,bokeh
0,13.03_bokeh


,grades
0,grades/getuserdetails/916/3
1,cohorts/24/grades
2,grades/getuserdetails/912/3
3,grades
4,grades/getuserdetails/918/3


,tree
0,7-classification/6.2-decision-trees
1,6.6.2_decision_tree
2,6.04.02_decisiontree
3,decision-trees


,remainder
0,04_navigating_the_filesystem
1,6.3_aquire
2,users/913
3,user/913
4,2.03.03_power
...,...
262,8.7_project
263,11.01_gettingstarted
264,fer
265,search


In [ ]:

question7




,page
search/search_index.json,19519
javascript-i,18983
toc,18297
java-iii,13733
html-css,13635
...,...
12-distributed-ml/6.1-prepare-part-1,1
12-distributed-ml/6.4-prepare-part-4,1
4-python,1
4-python/overview,1


In [ ]:









question7=question7.page.nsmallest(n=1,keep='all')




question7=question7.index


question7.sort()

TypeError: cannot sort an Index object in-place, use sort_values instead

In [ ]:

# for i in cols:
           
#           fig,axs=plt.subplots(1,3) 

#           print(i,axs,'\n')
#           kwargs={'family':'sans-serif'} 
#           fig.suptitle(t=f'{i}:\n [Hist w/ Kde] v. [Box] v. [Q-Q] ',**kwargs)
        
#           sns.histplot(x=i,data=df, kde=True,ax=axs[0])
#           print(i) 
#           sns.boxplot(x=i,data=df,ax=axs[1])
#           axs[2]=stats.probplot(df[i],dist='norm',plot=plt)


#           fig.set_figheight(8)
#           fig.set_figwidth(32)

#           plt.show()
#           print("\n\n\n")


# infodf=pd_DF_one_shot_info(df)

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(df.page)
